In [17]:
import pandas as pd
import gzip
import json
import re
import datetime
from math import floor
import numpy as np
from random import randint
from collections import Counter

In [18]:
from coinmarketcap import Coinmarketcap
from invest_strategies import InvestStrategies

# Initialize

In [19]:
market = Coinmarketcap("data/coinmarketcap_formatted.tsv")

# Strategies

## Top 1 coin

In [20]:
class OnlyBitcoin(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")

    def name(self):
        return "OnlyBitcoin"


## Top 5 in same proportion

In [21]:
class Top5Evenly(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=5, strategy="even")

    def name(self):
        return "Top 5 in same proportion"


## Top 5 by market cap

In [22]:
class Top5MarketCap(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=5, strategy="market_cap")

    def name(self):
        return "Top 5 by market cap"


## Top1 50%, Top2-5 50% by marketcap

In [23]:
class Top1_50_Top2_5_50(InvestStrategies):
    def buy(self, amount, date):
        top1_coins = self.buy_top_x(date, 1.0*amount/2, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        top2_5_coins = self.buy_top_x(date, 1.0*amount/2, top_x_first_pos=2, top_x_last_pos=5, strategy="even")
        coins = {**top1_coins, **top2_5_coins}
        return coins

    def name(self):
        return "Top1 50%, Top2-5 50% by marketcap"


## Top1 40%, TOP2 30%, Top3-5 20% evenly, Top3-10 10% evenly

In [24]:
class TopX(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            self.buy_top_x(date, 0.4*amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.3*amount, top_x_first_pos=2, top_x_last_pos=2, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.2*amount, top_x_first_pos=3, top_x_last_pos=5, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.1*amount, top_x_first_pos=3, top_x_last_pos=10, strategy="even")
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "Top1 40%, TOP2 30%, Top3-5 20% evenly, Top3-10 10% evenly"


## Top1 35%, TOP2 25%, Top3-5 20% evenly, Top3-10 20% evenly

In [25]:
class TopY(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            self.buy_top_x(date, 0.35*amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.25*amount, top_x_first_pos=2, top_x_last_pos=2, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.2*amount, top_x_first_pos=3, top_x_last_pos=5, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.2*amount, top_x_first_pos=3, top_x_last_pos=10, strategy="even")
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "Top1 35%, TOP2 25%, Top3-5 20% evenly, Top3-10 20% evenly"


# Configure

We will implement several strategies, and run them over several dates, from not before 2015-09-01 and not after 2017-04-30. We will run 1000 iterations with a minimun legth of 16 weeks (from first investment and last)

In [30]:
strategy_options = {
    'min_week_duration': 4,
    'max_week_duration': 4,
    'min_start_date': '2015-09-01'
}

# Run

In [31]:
print("Test parameters", json.dumps(strategy_options, indent=2))
iterations = None
for class_ in [OnlyBitcoin, Top5Evenly, Top5MarketCap, Top1_50_Top2_5_50, TopX, TopY]:
    inv = class_(market, strategy_options)
    results = inv.run_strategy(1000, balance_x_weeks=1)
    if iterations is None:
        iterations = results['iterations']
        print("iterations: %s\n\n" % iterations)
    inv.print_results(results)

Test parameters {
  "min_week_duration": 4,
  "max_week_duration": 4,
  "min_start_date": "2015-09-01"
}
iterations: 84




OnlyBitcoin:

Min return(2017-03-05 to 2017-04-02) 4 weeks: 0.77x.
Max return(2016-05-29 to 2016-06-26) 4 weeks: 1.44x.
Mean return(2016-10-23 to 2016-11-20) 4 weeks: 1.06x.


Top 5 in same proportion:

Min return(2016-07-24 to 2016-08-28) 4 weeks: 0.77x.
Max return(2017-04-16 to 2017-05-14) 4 weeks: 2.1x.
Mean return(2016-07-03 to 2016-07-31) 4 weeks: 1.03x.


Top 5 by market cap:

Min return(2016-06-19 to 2016-07-17) 4 weeks: 0.87x.
Max return(2017-04-16 to 2017-05-14) 4 weeks: 1.57x.
Mean return(2015-12-06 to 2016-01-03) 4 weeks: 1.06x.


Top1 50%, Top2-5 50% by marketcap:

Min return(2016-07-24 to 2016-08-28) 4 weeks: 0.81x.
Max return(2017-04-16 to 2017-05-14) 4 weeks: 1.79x.
Mean return(2016-04-24 to 2016-05-22) 4 weeks: 1.04x.


Top1 40%, TOP2 30%, Top3-5 20% evenly, Top3-10 10% evenly:

Min return(2016-07-24 to 2016-08-28) 4 weeks: 0.83x.
Max return(2017-0

In [27]:
""

''